In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Installing libraries

In [2]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

Requirement already up-to-date: gensim==4.0.0 in /usr/local/lib/python3.7/dist-packages (4.0.0)


In [3]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [4]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [5]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [6]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [7]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [8]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [9]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Albert Base AG News

In [10]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-ag-news")
tokenizer = AutoTokenizer("textattack/albert-base-v2-ag-news")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:08<00:00, 58.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpnl0y01an.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


# Framing fasttext attack

In [11]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [12]:
def printexample(attack):
  print(attack)
  from collections import deque

  import textattack
  import tqdm
  import time
  from IPython.display import display, HTML
  num_examples = 25
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

In [13]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Using custom data configuration default


textattack: Loading datasets dataset ag_news, split test.


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a. Subsequent calls will reuse this data.


In [14]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [15]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:27,  1.16s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:22<04:21, 11.38s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:33<04:09, 11.33s/it]

...

[Succeeded / Failed / Total] 14 / 9 / 25: 100%|██████████| 25/25 [06:21<00:00, 15.26s/it]

[Succeeded / Failed / Total] 14 / 10 / 26: : 26it [06:26, 14.86s/it]                     

[Succeeded / Failed / Total] 14 / 11 / 27: : 27it [06:33, 14.57s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 40.74% |
| Attack success rate:          | 56.0%  |
| Average perturbed word %:     | 20.39% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 567.12 |
+-------------------------------+--------+


textattack: Attack time: 393.28825640678406s


# Framing word2vec attack

In [16]:
from textattack.transformations import WordSwap

class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

In [17]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [18]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [19]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:27,  1.13s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:05<01:08,  2.98s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:09<01:11,  3.26s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [02:31<00:00,  6.05s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [02:34,  5.92s/it]                      

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [02:35,  5.76s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 7.11%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 244.4  |
+-------------------------------+--------+


textattack: Attack time: 155.4338150024414s


# Framing Glove attack

In [20]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

In [21]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [22]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [23]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:24,  1.01s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:09<01:51,  4.83s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:15<01:56,  5.30s/it]

...

[Succeeded / Failed / Total] 5 / 18 / 25: 100%|██████████| 25/25 [04:08<00:00,  9.95s/it]

[Succeeded / Failed / Total] 5 / 19 / 26: : 26it [04:13,  9.75s/it]                      

[Succeeded / Failed / Total] 5 / 20 / 27: : 27it [04:15,  9.47s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 74.07% |
| Attack success rate:          | 20.0%  |
| Average perturbed word %:     | 11.63% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 246.68 |
+-------------------------------+--------+


textattack: Attack time: 255.7977111339569s


In [24]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [25]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [26]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:17,  1.40it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:02<00:33,  1.46s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:04<00:33,  1.53s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [00:39<00:00,  1.58s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [00:40,  1.56s/it]                      

[Succeeded / Failed / Total] 9 / 16 / 27: : 27it [00:41,  1.55s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 59.26% |
| Attack success rate:          | 36.0%  |
| Average perturbed word %:     | 20.97% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 72.72  |
+-------------------------------+--------+


textattack: Attack time: 41.73041129112244s


In [27]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [28]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [29]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:17,  1.40it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:02<00:25,  1.12s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:03<00:26,  1.18s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [00:32<00:00,  1.30s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [00:33,  1.28s/it]                      

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [00:34,  1.27s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 11.27% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 63.6   |
+-------------------------------+--------+


textattack: Attack time: 34.2559916973114s


In [30]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [31]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [32]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:18,  1.32it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:02<00:30,  1.31s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:04<00:30,  1.40s/it]

...

[Succeeded / Failed / Total] 6 / 17 / 25: 100%|██████████| 25/25 [00:38<00:00,  1.53s/it]

[Succeeded / Failed / Total] 6 / 18 / 26: : 26it [00:39,  1.51s/it]                      

[Succeeded / Failed / Total] 6 / 19 / 27: : 27it [00:40,  1.49s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 70.37% |
| Attack success rate:          | 24.0%  |
| Average perturbed word %:     | 15.52% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 62.88  |
+-------------------------------+--------+


textattack: Attack time: 40.31463432312012s


In [33]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [34]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [35]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:03<01:14,  3.10s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:45<08:45, 22.84s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [01:09<08:31, 23.26s/it]

...

[Succeeded / Failed / Total] 12 / 11 / 25: 100%|██████████| 25/25 [11:31<00:00, 27.65s/it]

[Succeeded / Failed / Total] 12 / 12 / 26: : 26it [11:49, 27.30s/it]

[Succeeded / Failed / Total] 12 / 13 / 27: : 27it [12:07, 26.96s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 48.15% |
| Attack success rate:          | 48.0%  |
| Average perturbed word %:     | 17.01% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 1160.4 |
+-------------------------------+--------+


textattack: Attack time: 727.8773560523987s


In [36]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [37]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [38]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:02<01:08,  2.84s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:19<03:46,  9.86s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:32<03:54, 10.67s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [06:00<00:00, 14.40s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [06:12, 14.32s/it]                      

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [06:15, 13.91s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 7.11%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 729.08 |
+-------------------------------+--------+


textattack: Attack time: 375.65621876716614s


In [39]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [40]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [41]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:37,  1.55s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:24<04:43, 12.33s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:44<05:27, 14.89s/it]

...

[Succeeded / Failed / Total] 6 / 17 / 25: 100%|██████████| 25/25 [09:02<00:00, 21.71s/it]

[Succeeded / Failed / Total] 6 / 18 / 26: : 26it [09:20, 21.55s/it]

[Succeeded / Failed / Total] 6 / 19 / 27: : 27it [09:25, 20.93s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 70.37% |
| Attack success rate:          | 24.0%  |
| Average perturbed word %:     | 14.99% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 704.12 |
+-------------------------------+--------+


textattack: Attack time: 565.2332630157471s


In [42]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [43]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [44]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:22,  1.08it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:03<00:37,  1.63s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:05<00:38,  1.76s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:44<00:00,  1.80s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:46,  1.77s/it]                      

[Succeeded / Failed / Total] 7 / 18 / 27: : 27it [00:47,  1.76s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 66.67% |
| Attack success rate:          | 28.0%  |
| Average perturbed word %:     | 21.34% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 74.24  |
+-------------------------------+--------+


textattack: Attack time: 47.43352651596069s


In [45]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [46]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [47]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:20,  1.15it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:02<00:29,  1.28s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:04<00:30,  1.38s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [00:37<00:00,  1.49s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [00:38,  1.46s/it]                      

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [00:39,  1.45s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 15.44% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 64.96  |
+-------------------------------+--------+


textattack: Attack time: 39.119497299194336s


In [48]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [49]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [50]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:22,  1.07it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:03<00:35,  1.56s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:05<00:38,  1.74s/it]

...

[Succeeded / Failed / Total] 5 / 18 / 25: 100%|██████████| 25/25 [00:46<00:00,  1.86s/it]

[Succeeded / Failed / Total] 5 / 19 / 26: : 26it [00:47,  1.84s/it]                      

[Succeeded / Failed / Total] 5 / 20 / 27: : 27it [00:49,  1.82s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 74.07% |
| Attack success rate:          | 20.0%  |
| Average perturbed word %:     | 13.8%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 61.48  |
+-------------------------------+--------+


textattack: Attack time: 49.22118616104126s


In [51]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [52]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [53]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:04<01:53,  4.72s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:59<11:26, 29.84s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [01:37<11:55, 32.51s/it]

...

[Succeeded / Failed / Total] 13 / 10 / 25: 100%|██████████| 25/25 [14:19<00:00, 34.39s/it]

[Succeeded / Failed / Total] 13 / 11 / 26: : 26it [14:25, 33.28s/it]

[Succeeded / Failed / Total] 13 / 12 / 27: : 27it [14:46, 32.83s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 13      |
| Number of failed attacks:     | 12      |
| Number of skipped attacks:    | 2       |
| Original accuracy:            | 92.59%  |
| Accuracy under attack:        | 44.44%  |
| Attack success rate:          | 52.0%   |
| Average perturbed word %:     | 14.64%  |
| Average num. words per input: | 45.04   |
| Avg num queries:              | 1294.44 |
+-------------------------------+---------+


textattack: Attack time: 886.5158219337463s


In [54]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [55]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [56]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:04<01:39,  4.16s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:28<05:24, 14.12s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:53<06:35, 17.99s/it]

...

[Succeeded / Failed / Total] 6 / 17 / 25: 100%|██████████| 25/25 [09:16<00:00, 22.27s/it]

[Succeeded / Failed / Total] 6 / 18 / 26: : 26it [09:31, 22.00s/it]

[Succeeded / Failed / Total] 6 / 19 / 27: : 27it [09:41, 21.52s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 70.37% |
| Attack success rate:          | 24.0%  |
| Average perturbed word %:     | 20.49% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 913.16 |
+-------------------------------+--------+


textattack: Attack time: 581.0925552845001s


In [57]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [58]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [59]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:05<02:12,  5.51s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:27<05:18, 13.84s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [01:09<08:27, 23.09s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [14:47<00:00, 35.51s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [15:05, 34.83s/it]

[Succeeded / Failed / Total] 7 / 18 / 27: : 27it [15:22, 34.17s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 66.67% |
| Attack success rate:          | 28.0%  |
| Average perturbed word %:     | 16.3%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 769.96 |
+-------------------------------+--------+


textattack: Attack time: 922.6103792190552s
